In [1]:
import sys, os
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())))
try:
    print(curr_dir)
except:
    curr_dir = os.path.dirname(os.path.abspath(os.getcwd()))
    os.chdir(curr_dir)
    print(curr_dir)

import json
import numpy as np
import torch
from stable_baselines3 import PPO, DDPG
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.utils import safe_mean
from typing import *
from envs import BlendEnv, flatten_and_track_mappings, reconstruct_dict
from envs_omni import *
from utils import *

c:\Users\adame\OneDrive\Bureau\CODE\BlendingRL


In [2]:
env = BlendEnv()

In [2]:
layout = "simple"
connections, action_sample = get_jsons(layout)
sources, blend, demands = get_sbp(connections)
tau0   = {s: [10, 10, 10, 0, 0, 0] for s in sources}
delta0 = {d: [0, 0, 0, 10, 10, 10] for d in demands}

In [3]:
from gymnasium import spaces

class ExampleEnv(CMDP):
    _support_envs = ['Example-v0', 'Example-v1']  # Supported task names

    need_auto_reset_wrapper = True  # Whether `AutoReset` Wrapper is needed
    need_time_limit_wrapper = True  # Whether `TimeLimit` Wrapper is needed

    def __init__(self, env_id: str, **kwargs) -> None:
        self._count = 0
        self._num_envs = 1
        self._observation_space = spaces.Box(low=-1.0, high=1.0, shape=(3,))
        self._action_space = spaces.Box(low=-1.0, high=1.0, shape=(2,))

    def set_seed(self, seed: int) -> None:
        random.seed(seed)

    def reset(
        self,
        seed = None,
        options  = None,
    ):
        if seed is not None:
            self.set_seed(seed)
        obs = torch.as_tensor(self._observation_space.sample())
        self._count = 0
        return obs, {}

    @property
    def max_episode_steps(self) -> None:
        """The max steps per episode."""
        return 10

    def render(self):
        pass

    def close(self) -> None:
        pass

    def step(
        self,
        action: torch.Tensor):
        self._count += 1
        obs = torch.as_tensor(self._observation_space.sample())
        reward = 2 * torch.as_tensor(random.random())
        cost = 2 * torch.as_tensor(random.random())
        terminated = torch.as_tensor(random.random() > 0.9)
        truncated = torch.as_tensor(self._count > 10)
        return obs, reward, cost, terminated, truncated, {'final_observation': obs}

In [ ]:
env = BlendEnv_Omni(env_id="Blend-base")

In [6]:
env.reset(seed=0)
while True:
    action = env.action_space.sample()
    obs, reward, terminated, truncated, info = env.step(action)
    print('-' * 20)
    print(f'obs: {obs}')
    print(f'reward: {reward}')
    # print(f'cost: {cost}')
    print(f'terminated: {terminated}')
    print(f'truncated: {truncated}')
    print('*' * 20)
    if terminated or truncated:
        break
env.close()

--------------------
obs: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. 10. 30.  0.  0. 10. 30. 15. 15.  0.  0. 15. 15.  0.  0. 15. 15.
  0.  0. 15. 15.  0.  0.  0.  0.  1.]
reward: -50982.07370586395
terminated: False
truncated: False
********************
--------------------
obs: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0. 10. 30. 15. 15.  0.  0. 15. 15.  0.  0. 15. 15.  0.  0. 15. 15.
  0.  0.  0.  0.  0.  0.  0.  0.  2.]
reward: -99067.39359989166
terminated: False
truncated: False
********************
--------------------
obs: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0. 15. 15.  0.  0. 15. 15.  0.  0. 15. 15.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  3.]
reward: -142723.81819031236
terminated: False
truncated: False
********************
--------------------
obs: [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0. 15. 